In [3]:
!pip install openai
import os
from openai import OpenAI

os.environ["PERPLEXITY_API_KEY"] = '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' # perplexity API key 

client = OpenAI(
    api_key=os.environ["PERPLEXITY_API_KEY"],
    base_url="https://api.perplexity.ai"
)

def perplexity(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    try:
        response = client.chat.completions.create(
            model="sonar",  
            messages=messages
        )
        return response.choices[0].message.content.strip(), []  
    except Exception as e:
        print("Error:", e)
        return "", []


In [5]:
### FOR THE URLs ###

import pandas as pd

df = pd.read_csv("10k_movies.csv")
first_100 = df.head(100)

system_prompt = (
    "You are going to extract URLs from letterboxd.com based on the CSV file provided. "
    "Each entry contains two columns: 'title' and 'release_date'. Use these to identify the correct movie. "
    "Only return a valid Letterboxd URL that leads directly to the movie’s page. "
    "Do not return any other URLs (such as person profiles, lists, or search results). "
    "Be strict about the release year. Do not return movies from a different year. "
    "If you're not sure, return nothing."
)

urls = []

for index, row in first_100.iterrows():
    title = row['title']
    release_date = row['release_date']

    prompt = f"Give me the Letterboxd URL for the movie: {title} ({release_date})"
    result, _ = perplexity(prompt, system_prompt)

    # Filter out bad results
    if result and "letterboxd.com/film/" in result:
        # Grab the first matching line that contains a URL
        for line in result.splitlines():
            if "letterboxd.com/film/" in line:
                url = line.strip().split()[0]  # Get the URL only
                if url.startswith("http"):
                    urls.append(url)
                    break

# Write all URLs to a text file
with open("letterboxd_urls.txt", "w") as f:
    for url in urls:
        f.write(url + "\n")

print("Saved clean URL list to 'letterboxd_urls.txt'")

Saved clean URL list to 'letterboxd_urls.txt'
